In [1]:
import pandas as pd
data=pd.read_csv('Products.csv')

In [2]:
filtered_data = data[data['main_category'] == 'home & kitchen']
filtered_data['sub_category'].unique()

array(['All Home & Kitchen', 'Bedroom Linen', 'Furniture',
       'Garden & Outdoors', 'Home Décor', 'Home Furnishing',
       'Home Improvement', 'Home Storage', 'Indoor Lighting',
       'Kitchen & Dining', 'Kitchen Storage & Containers',
       'Sewing & Craft Supplies'], dtype=object)

In [3]:
filtered_data['tags'] = filtered_data['main_category'] + ' ' + filtered_data['sub_category'] + ' ' + filtered_data['ratings'].astype(str) + ' ' + filtered_data['discount_price'] + ' ' + filtered_data['actual_price']

# Optionally, you can drop the original columns if needed
df = filtered_data.drop(columns=['main_category', 'sub_category', 'ratings', 'discount_price', 'actual_price','image','link','no_of_ratings'])


C:\Users\SYED AZIZ AHMED\AppData\Local\Temp\ipykernel_16352\3245301995.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['tags'] = filtered_data['main_category'] + ' ' + filtered_data['sub_category'] + ' ' + filtered_data['ratings'].astype(str) + ' ' + filtered_data['discount_price'] + ' ' + filtered_data['actual_price']


In [4]:
df = df.reset_index(drop=True)

In [5]:
df

,name,tags
0,Godrej aer Matic Kit (Machine + 1 Refill) - Au...,home & kitchen All Home & Kitchen 4.1 ₹466 ₹56...
1,"Presto! Oxo-Biodegradable Garbage Bags, Large ...",home & kitchen All Home & Kitchen 4.3 ₹349 ₹480
2,Happilo 100% Natural Premium 200g Whole Cashew...,home & kitchen All Home & Kitchen 4.2 ₹199 ₹335
3,beatXP Kitchen Scale Multipurpose Portable Ele...,"home & kitchen All Home & Kitchen 3.7 ₹399 ₹1,999"
4,"Dettol Antiseptic Liquid for First Aid , Surfa...",home & kitchen All Home & Kitchen 4.6 ₹342 ₹364
...,...,...
13752,"Glue Water Dots Set of 50 for Art, DIY, Gift P...",home & kitchen Sewing & Craft Supplies 3.9 ₹19...
13753,Balar Lace Border Multi Color Mirror Work Embr...,home & kitchen Sewing & Craft Supplies 4.1 ₹30...
13754,M.G ENTERPRISE 100% Acrylic Wool Multi Royal (...,home & kitchen Sewing & Craft Supplies 5 ₹362 ...
13755,Goodwill Tech™ Recycled Newspaper Pencils Eco-...,home & kitchen Sewing & Craft Supplies 5 ₹399 ...


In [6]:
df.describe()

,name,tags
count,13757,13757
unique,13040,12940
top,BSB HOME Premium Cotton Elastic Fitted Bedshee...,home & kitchen Home Furnishing 4.1 ₹299 ₹999
freq,12,10


In [7]:
data=df

In [8]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from scipy.sparse import csr_matrix
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np


# Convert 'tags' to lowercase
data['tags'] = data['tags'].str.lower()

# Tokenization and stopword removal
stop_words = set(stopwords.words('english'))

def preprocess_tags(tags):
    words = word_tokenize(tags)
    filtered_words = [word for word in words if word not in stop_words]
    return ' '.join(filtered_words)

data['tags'] = data['tags'].apply(preprocess_tags)

# Initialize a CountVectorizer with a limited number of features
count_vectorizer = CountVectorizer(max_features=1000)

# Vectorization (sparse matrix)
count_matrix = count_vectorizer.fit_transform(data['tags'])

# Calculate cosine similarity
cosine_sim = cosine_similarity(count_matrix, count_matrix)

# Convert the similarity matrix to a sparse matrix
sparse_cosine_sim = csr_matrix(cosine_sim)

print("Cosine Similarity Matrix Shape:", sparse_cosine_sim.shape)


Cosine Similarity Matrix Shape: (13757, 13757)


In [9]:
filtered_data.to_csv('HomeandKitchen.csv')

In [10]:
import pickle
with open('HomeandKitchen.pkl','wb') as file:
    pickle.dump(cosine_sim,file)

In [13]:
import pickle
import pandas as pd
data=pd.read_csv('HomeandKitchen.csv')
with open('HomeandKitchen.pkl', 'rb') as file:
    cosine_si = pickle.load(file)
def recommend(items):
    index = data[data['name'] == items].index[0]
    distances = sorted(list(enumerate(cosine_si[index])), reverse=True, key=lambda x: x[1])
    recommended_items = [data.iloc[i[0]]['name'] for i in distances[1:6]]
    return recommended_items

recommended_items = recommend('PHILIPS Hue Gen 3 Smart Light E27 Color Ambiance 9W Bulb, Bluetooth & Zigbee Compatible Hue Bridge Optional, Compatible wi...')
for items in recommended_items:
    print(items)


PHILIPS Hue Gen 3 Smart Light E27 Color Ambiance 9W Bulb, Bluetooth & Zigbee Compatible Hue Bridge Optional, Compatible wi...
PHILIPS H1 Ultinon Essential G2 LED Lamp 6000K Luxeon (Pure White, 2 Pieces)
SYSKA SSK-RDL-R-5W-6K 5-Watt Round LED Slim Recessed Panel Lights (Pack of 6, Cool Day Light)
WAVE SHOP Plastic Touch Lamp Night Light, Multicolour, Pack of 2
Hopdezyo E27 LED Italian Crafter Home Decor Floor Lamp for Living Room Bedroom Office, Holder & Lamp, Jute Shade Pack of 1
